# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Set path for file
datapath = os.path.join("..", "WeatherPy","output_data","cities.csv")

# Skip importing the first column in the DataFrame (the index in this case)
cities_df = pd.read_csv(datapath, usecols=range(1,10))
cities_df.head()

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Rikitea,PF,-23.12,-134.97,21.27,79,0,8.73,1595796109
1,Hikari,JP,33.95,131.95,24.00,94,75,4.10,1595796109
2,Mataura,NZ,-46.19,168.86,3.89,83,0,1.34,1595796110
3,Finnsnes,NO,69.23,17.98,16.00,63,20,8.70,1595796110
4,Port Elizabeth,ZA,-33.92,25.57,13.00,87,62,3.10,1595796110


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Long"]]

# Fill NaN values and convert to float
rating = cities_df["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(35, -80), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=max(rating),
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_df = cities_df.loc[(cities_df["Max Temp"] >= 18) & (cities_df["Max Temp"] <= 24.5) & 
                                   (cities_df["Humidity"] >= 30) & (cities_df["Humidity"] <= 50) &
                                   (cities_df["Wind Speed"] <= 8)
                                   ]
ideal_df.head()
# Index 95 has a 'NaN' value in the 'Country' field, but I don't think that merits discarding the data

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
90,Yellowknife,CA,62.46,-114.35,22.00,40,20,3.60,1595796120
95,Lüderitz,NaN,-26.65,15.16,22.00,43,0,4.10,1595796121
155,Mehamn,NO,71.04,27.85,20.00,49,0,3.60,1595796129
210,Norman Wells,CA,65.28,-126.83,23.00,40,75,5.10,1595795720
275,Santa Fe,AR,-31.63,-60.70,18.33,32,0,2.24,1595796128


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# Add "Hotel Name" column to hotel_df
hotel_df = ideal_df.copy()
hotel_df["Hotel Name"] = ""

In [13]:
# Setup the url and search parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    lat = str(row["Lat"])
    long = str(row["Long"])
    params["location"] = lat + ", " + long

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:      
        hotel_df.loc[index, 'Hotel Name'] = results[0]["name"]
        
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "None"

In [14]:
hotel_df

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
90,Yellowknife,CA,62.46,-114.35,22.00,40,20,3.60,1595796120,The Explorer Hotel
95,Lüderitz,NaN,-26.65,15.16,22.00,43,0,4.10,1595796121,LÜDERITZ NEST HOTEL
155,Mehamn,NO,71.04,27.85,20.00,49,0,3.60,1595796129,Mehamn Arctic Hotel AS
210,Norman Wells,CA,65.28,-126.83,23.00,40,75,5.10,1595795720,Heritage Hotel
275,Santa Fe,AR,-31.63,-60.70,18.33,32,0,2.24,1595796128,Conquistador Hotel
301,Sotnikovskoye,RU,45.00,43.78,23.56,45,24,5.03,1595796147,None
305,Benavente,ES,42.00,-5.68,24.00,41,0,0.50,1595796147,Parador de Benavente
312,Lodja,CD,-3.48,23.43,23.84,43,0,0.50,1595796148,None
329,Fort Nelson,CA,58.81,-122.70,23.00,49,75,1.00,1595796063,The Fort Nelson Hotel
366,Bukama,CD,-9.20,25.85,18.92,38,40,1.36,1595796155,None


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Country</dt><dd>{Country}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Long"]]

In [16]:
# Create a map using state centroid coordinates to set markers
marker_locations = hotel_df[['Lat', 'Long']]

fig = gmaps.figure(center=(35, -80), zoom_level=2, map_type='HYBRID')

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=max(rating),
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Add hotel markers
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))